In [4]:
from time import perf_counter
import numpy as np,os
from imblearn.under_sampling import InstanceHardnessThreshold, EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [6]:
#ENN already done in Noisy.csv, but kind_sel='mode' here, so here aggressive than before
imbs = {'ENN':EditedNearestNeighbours(n_neighbors=5,kind_sel='mode'), 
       'SmoteEnc':SMOTEENN(),'IHThreshold':InstanceHardnessThreshold()}
models = {}
for im,samp in imbs.items():
    for c,clf in CLFS.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',clf)])
    
models.keys(),len(models)

(dict_keys([('ENN', 'dt'), ('ENN', 'lr'), ('ENN', 'nb'), ('ENN', 'svm'), ('ENN', 'knn'), ('ENN', 'rf'), ('SmoteEnc', 'dt'), ('SmoteEnc', 'lr'), ('SmoteEnc', 'nb'), ('SmoteEnc', 'svm'), ('SmoteEnc', 'knn'), ('SmoteEnc', 'rf'), ('IHThreshold', 'dt'), ('IHThreshold', 'lr'), ('IHThreshold', 'nb'), ('IHThreshold', 'svm'), ('IHThreshold', 'knn'), ('IHThreshold', 'rf')]),
 18)

In [7]:
path = "Noisy.csv"
cols = pd.MultiIndex.from_product([imbs.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)

In [ ]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)